# Formalia:

Please read the [assignment overview page](https://github.com/suneman/socialgraphs2025/wiki/Assignments) carefully before proceeding. This page contains information about formatting (including formats etc), group sizes, and many other aspects of handing in the assignment. 

_If you fail to follow those simple instructions, it will negatively impact your grade!_

**Due date and time**: The assignment is due on Tuesday November 4th, 2025 at 23:55. Hand in your IPython notebook file (with extension `.ipynb`) via DTU Learn

In the exercises below, I describe the exercises in a general way. Drawing in the right parts of the exercises is part of the assignment. (That way we're helping you get a little bit more ready for the Final Project, where you have to decide what information to include in your report and analysis). 

# Part 1: Analyze the network

The questions in this part are based on Lecture 5.

* Present an analysis/description of the network of bands/artists using tools from Lecture 5. Imagine that you have been tasked with presenting the important facts about the network to an audience who knows about network science, but doesn't know about this particular network.
   - It's OK to also use basic concepts like degree distributions (even though they're from week 4) in your analysis. That way you can make the analysis a standalone, coherent thing.
   - I would like you to include concepts like centrality and assortativity in your analysis.
   - Use a network backbone in your analysis.
   - In addition to standard distribution plots (e.g. degree distributions, etc), your analysis should also include at least one network visualization (but it doesn't have to display the entire network, you can also visualize a network backbone).
   - **Note**: As I write above, an important part of the exercise consists is *selecting the right elements of the lecture* to create a meaningful analysis. So don't solve this part by going exhaustive and just calculating everything you can think of in one massive analysis. Try to focus on using what you've learned to characterize the network. 

# Part 2: Genres and communities and plotting 

The questions below are based on Lecture 7, part 2.

* Write about genres and modularity.
* Detect the communities, discuss the value of modularity in comparison to the genres.
* Calculate the matrix $D$ and discuss your findings.
* Plot the communities and comment on your results.

# Part 3: TF-IDF to understand genres and communities 

The questions below  are based on Lecture 7, part 2, 4, 5, 6 (and a little bit on part 3).

* Explain the concept of TF-IDF in your own words and how it can help you understand the genres and communities.
* Calculate and visualize TF-IDF for the genres and communities.
* Use the matrix $D$ (Lecture 7, part 2) to dicusss the difference between the word-clouds between genres and communities.

# Part 4: Sentiment of the artists and communities

The questions below are based on Lecture 8

* Calculate the sentiment of the band/artist pages (it is OK to work with the sub-network of artists-with-genre) and describe your findings using stats and visualization, inspired by the first exercise of week 8.
* Discuss the sentiment of the communities. Do the findings using TF-IDF during Lecture 7 help you understand your results?

In [ ]:
# Imports
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, warnings
import os
import re
import ast
import json
import random
import statistics
from scipy.stats import pearsonr, binom
from pathlib import Path
from typing import List, Dict, Optional
from collections import Counter, defaultdict

random.seed(42)

: 

In [ ]:
# Create Graph
G = nx.DiGraph()

files = os.listdir("Musicians")

for file in files:
    filepath = "Musicians/" + file
    with open(filepath, "r") as f:
        G.add_node(file.replace(".txt", ''), length_of_content = len(f.read().split()))

for file in files:
    filepath = "Musicians/" + file
    with open(filepath, "r") as f:
        filecontent = f.read()

        links = re.findall(r'\[\[([^|\]#]+)(?:#[^\]]*)?(?:\|([^\]]+))?\]\]', filecontent)

        for link, _ in links:
            link = link.replace("\\", "")
            link = link.replace('/', ' ')
            if link in list(G.nodes):
                G.add_edge(file.replace(".txt", ''), link.replace('/', ' '))

noConnections = [node for node in G.nodes if (G.out_degree(node) == 0 and G.in_degree(node) == 0)]

G.remove_nodes_from(noConnections)

largest_wcc = max(nx.weakly_connected_components(G), key=len)

GL = G.subgraph(largest_wcc).copy()

G_und = GL.to_undirected() 

In [ ]:
def normalize_genre(genre_text: str) -> Optional[str]:
    """
    Cleans, lowercases, and normalizes a single genre string.
    """
    # Clean and lowercase
    g = genre_text.lower().strip()
    
    if not g:
        return None
    
    g = g.replace('-', ' ')
        
    # Normalize "rock'n'roll", "rock & roll", etc.
    # This rule is intentionally broad to catch variations.
    if 'rock' in g and ('roll' in g or '’n’' in g or '&' in g or "'n'" in g):
        return 'rock and roll'
    
    g = g.replace('&', 'and')
    
    if 'early' in g:
        g = g.replace(' (early)', '')
    
    if 'later' in g:
        g = g.replace(' (later)', '')
        
        
    # Add any other normalization rules here
    # For example:
    # if g == 'r&b':
    #     return 'r&b'
        
    return g

def extract_all_genres(directory_path: str) -> Dict[str, List[str]]:
    """
    Extracts genres for all artists from wikitext files in a directory.
    
    Args:
        directory_path: The path to the folder containing artist .txt files.

    Returns:
        A dictionary mapping artist names to a list of their genres.
    """
    artist_genres_map: Dict[str, List[str]] = {}
    p = Path(directory_path)
    
    if not p.is_dir():
        print(f"Error: Directory not found at {directory_path}")
        return {}

    # Define regex patterns
    # 1. Finds the main infobox
    infobox_re = re.compile(
        r"\{\{Infobox musical artist[\s\S]*?^\}\}", 
        re.MULTILINE | re.IGNORECASE
    )
    
    # 2. Finds the genre line within the infobox
    # This version stops when it sees the *next parameter* (e.g. "| label =")
    # or the end of the infobox, rather than just any line starting with "|".
    genre_line_re = re.compile(
        r"\| *genre *= *([\s\S]*?)\n *(?=\| *[a-z_ ]+ *=|^\}\})", 
        re.MULTILINE | re.IGNORECASE
    )
    
    # 3-7. Cleaning regex patterns
    link_display_re = re.compile(r"\[\[[^|\]]+\|([^\]]+)\]\]") # [[link|display]] -> display
    link_re = re.compile(r"\[\[([^\]]+)\]\]")                 # [[link]] -> link
    # NEW: Separator for adjacent links like ]] [[
    link_separator_re = re.compile(r"\]\](\s+)\[\[")
    
    list_template_re = re.compile(                           # {{hlist|a|b}} -> a|b
        r"\{\{(?:hlist|flatlist|unbulleted list)\s*\|([\s\S]*?)\}\}", 
        re.IGNORECASE
    )
    other_template_re = re.compile(r"\{\{[^}]+\}\}")         # {{template}} -> ""
    comment_re = re.compile(r"<!--[\s\S]*?-->")              # <!-- comment --> -> ""
    ref_re = re.compile(r"<ref[\s\S]*?(/>|</ref>)", re.IGNORECASE) # <ref ... /> -> ""
    # UPDATED: Add <br> variants to the separator list
    separator_re = re.compile(r"[|\n*]|<br */?>", re.IGNORECASE) # |, \n, *, <br> -> ,

    for file_path in p.glob('*.txt'):
        artist_name = file_path.stem
        wikitext = ""
        
        try:
            # Read the raw content
            raw_content = file_path.read_text(encoding='utf-8')
            
            # Try to parse as the 'ast' structure from your example
            try:
                data = ast.literal_eval(raw_content)
                page_id = list(data.keys())[0] # Get the first (only) page ID
                wikitext = data[page_id]['revisions'][0]['*']
            except (ValueError, SyntaxError, KeyError, IndexError, TypeError):
                # Fallback: assume the file is just raw wikitext
                wikitext = raw_content
                
            if not wikitext:
                print(f"No wikitext found for {artist_name}")
                continue

            # 1. Find infobox
            infobox_match = infobox_re.search(wikitext)
            if not infobox_match:
                # print(f"No infobox found for {artist_name}")
                continue
            
            infobox_text = infobox_match.group(0)

            # 2. Find genre line
            genre_match = genre_line_re.search(infobox_text)
            if not genre_match:
                # print(f"No genre line found for {artist_name}")
                continue
                
            content = genre_match.group(1)

            # --- Start cleaning the extracted genre content ---
            # The order of these operations is critical.
            
            # 0. Replace HTML entities like &nbsp;
            content = content.replace('&nbsp;', ' ')
            
            # 1. Remove HTML comments <!-- ... -->
            content = comment_re.sub("", content)
            # 2. Remove <ref> tags (which can contain nested templates)
            content = ref_re.sub("", content)
            
            # 3. NEW: Insert a separator (pipe) between adjacent links: ]] [[ -> ]] | [[
            # This handles cases like [[hard rock]] [[arena rock]]
            content = link_separator_re.sub(r"]]|\1[[", content)
            
            # 4. Simplify hlist/flatlist templates: {{hlist|a|b|c}} -> a|b|c
            content = list_template_re.sub(r"\1", content)
            # 5. Remove any other simple templates {{template}} -> ""
            content = other_template_re.sub("", content)
            # 6. Resolve links: [[link|display]] -> display
            content = link_display_re.sub(r"\1", content)
            # 7. Resolve links: [[link]] -> link
            content = link_re.sub(r"\1", content)
            # 8. Replace common separators (pipe, newline, asterisk, <br>) with commas
            content = separator_re.sub(",", content)
            
            # 9. Final cleanup: strip whitespace and any trailing template braces
            content = content.strip().rstrip('}')
            # --- Done cleaning ---

            # 10. Split by comma and process each potential genre
            potential_genres = content.split(',')
            genres_list = []
            
            for g_text in potential_genres:
                # Extra cleanup for trailing braces on individual items
                g_clean = g_text.strip().rstrip('}')
                
                # Skip any empty strings or leftover template/HTML bits
                # ADDED: Filter out template parameters (like 'italic=no' by checking for '=')
                # ADDED: Filter out very short items (like 'de' by checking length > 2)
                if (g_clean and
                    not g_clean.startswith('<') and
                    not g_clean.startswith('{') and
                    'class=' not in g_clean and
                    '=' not in g_clean and
                    len(g_clean) > 2):
                    
                    normalized = normalize_genre(g_clean)
                    if normalized:
                        genres_list.append(normalized)

            # 11. Add to map if we found any genres
            if genres_list:
                # Remove duplicates and sort
                unique_genres = list(set(genres_list))
                artist_genres_map[artist_name] = unique_genres

        except Exception as e:
            print(f"Error processing {artist_name} ({file_path.name}): {e}")
            
    return artist_genres_map


genre_dict = extract_all_genres("Musicians")

# Part 1: Analyze the network

**We want to present the some important facts about the network that results from the links between pages from rock musicians on wikipedia.**

**We start of with some basic facts on the network:**

**What is the number of nodes?**

**What is the number of edges?**

**What are the average, median, mode, minimum and maximum value of the in-degree and of the out-degree?**

In [ ]:
in_degrees = dict(GL.in_degree())
out_degrees = dict(GL.out_degree())

nodes = list(GL.nodes())
x_out = [out_degrees[n] for n in nodes]
y_in  = [in_degrees[n] for n in nodes]

def compute_stats(values):
    return {
        "average": sum(values) / len(values),
        "median": statistics.median(values),
        "mode": statistics.mode(values),
        "min": min(values),
        "max": max(values) ,
    }

in_stats = compute_stats(y_in)
out_stats = compute_stats(x_out)

print(f"The network has {len(GL.nodes)} nodes and {len(GL.edges)} edges.")
print(f"The average degree of the nodes is {in_stats["average"]:.2f}.")
print(f"The median of the in-degrees is {in_stats["median"]}, the mode is {in_stats["mode"]}, the minimum is {in_stats["min"]}, and the maximum {in_stats["max"]}.")
print(f"If we compare this to the out-degrees, we notice that the out-degree distribution seems to be different, since the median is {out_stats["median"]}, the mode is {out_stats["mode"]}, the minimum is {out_stats["min"]}, and the maximum {out_stats["max"]}.")

**To investigate this further, we will now plot the in- and out-degree distribution**

In [ ]:
number_bins = 50

plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.hist([x for x in y_in], bins=number_bins)
plt.xlabel("In-degree")
plt.ylabel("Frequency")
plt.title("In-degree Distribution Musicians")

plt.subplot(1,2,2)
plt.hist([x for x in x_out], bins=number_bins)
plt.xlabel("Out-degree")
plt.ylabel("Frequency")
plt.title("Out-degree Distribution Musicians")
plt.tight_layout()
plt.show()

Most musicians have an in-degree of 0 or a very small one, and a few musicians have a very high in-degree. The out-degree follows more of a normal distribution. Our theory for why this is the case is that many musicians draw inspiration from a very small group of highly influential musicians. 

**To investige this further we will now look into the degree distributions of the individual musicians in more detail.**

**First we will create a scatter plot, where each point is a musician, and the axes show in- versus out-degree**

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(x_out, y_in, alpha=0.6, s=30)

for i, node in enumerate(nodes):
    if x_out[i] > 60 or y_in[i] > 60:
        plt.text(x_out[i]+0.2, y_in[i]+0.2, str(node), fontsize=8, alpha=0.7)

plt.xlabel("Out-degree")
plt.ylabel("In-degree")
plt.title("Scatter plot of In-degree vs Out-degree")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

This scatter plot shows the in- versus out-degree for each musician. To avoid too much overlapping of names, only the nodes that have an in- or out-degree over 60 have a label.

We notice that there is indeed some few musicians with a very high in-degree, who are all well known, which supports our theory from above.

We also notice that most musicians have an in- and out-degree both less than 20. However, we can also see not two much due to the many nodes in that area. 

**So we will create a heatmap in the next step, that zooms in on just the in-degree $[0,20]$ and out-degree $[0,20]$ area of the plot.**

In [ ]:
plt.hist2d(
    x_out,
    y_in,
    bins=[20, 20],
    range=[[0, 20], [0, 20]],
    cmap='viridis'
)

plt.colorbar(label="Number of musicians")
plt.xlabel("Out-degree")
plt.ylabel("In-degree")
plt.title("Heatmap of In-degree vs Out-degree (zoomed 0-20)")

plt.show()

In this heatmap we can see again support for our theory. We see that the majority of musicians has lower in-degree then out-degree.

**To further support our theory, we will now calculate the pearson correlation between the length of the wikipedia articals and the in-degree, since longer wikipedia articiles could also be a sign of famous characters**

In [ ]:
lengths = [G.nodes[n].get('length_of_content', 0) for n in G.nodes()]
in_degrees_pearson = [G.in_degree(n) for n in G.nodes()]

pearson_in  = pearsonr(lengths, in_degrees_pearson)
print(f"The pearson correlation is {pearson_in.correlation:.4f}, which is a very high correlation, which again supports our theory")

In [ ]:
degree_centrality = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G)

degree_centrality_list = [degree_centrality[n] for n in GL.nodes()]
betweenness_list = [betweenness[n] for n in GL.nodes()]

plt.figure(figsize=(20, 8))
plt.subplot(1,2,1)
plt.xlabel("Degree centrality")
plt.ylabel("Betweenness centrality")
plt.title("Scatter plot of degree centrality vs betweenness centrality of nodes")
plt.grid(True, linestyle="--", alpha=0.5)
plt.scatter(degree_centrality_list, betweenness_list, alpha=0.6, s=30)

scc = max(nx.strongly_connected_components(G), key=len)
G_scc = G.subgraph(scc).copy()
out_eig = nx.eigenvector_centrality(G_scc)

degree_centrality_scc = nx.degree_centrality(G_scc)
degree_centrality_list_scc = [degree_centrality[n] for n in G_scc.nodes()]
out_eig_list = [out_eig[n] for n in G_scc.nodes()]

plt.subplot(1,2,2)
plt.xlabel("Degree centrality")
plt.ylabel("Eigenvector centrality")
plt.title("Scatter plot of degree centrality vs betweenness centrality of nodes")
plt.grid(True, linestyle="--", alpha=0.5)
plt.scatter(degree_centrality_list_scc, out_eig_list, alpha=0.6, s=30)
plt.show()


Next, we want to look at associativity in our network, i.e. do nodes with similar attributes tend to connect to one another.

**We will test this with both the degree and the length of content as attributes on the undirected version of the network**

In [ ]:
G_und_with_degree_attribute = G_und.copy()

degrees = dict(G_und_with_degree_attribute.degree())

nx.set_node_attributes(G_und_with_degree_attribute, degrees, "degree")

deg_asso = nx.attribute_assortativity_coefficient(G_und_with_degree_attribute, "degree")
loc_asso = nx.attribute_assortativity_coefficient(G_und_with_degree_attribute, "length_of_content")

print(f"Both coefficients, for the degree assortativity {deg_asso:.4f} as well as for the length of content assortativity {loc_asso:.4f}, are close to 0. This means the network is non-assortative for these attributes.")
print("A reason for that could be, that musicians who are not as famous, meaning lower degree and shorter articles, tend to connect to famous musicians, where they maybe got inspiration, as well as between themselves, maybe because of collaberations.")
print("The famous musicans are then connected to the unfamous ones because the graph is undirected and they also colaborate or are influenced by other famous musicians.")

In [ ]:
# Code from https://www.michelecoscia.com/?page_id=287
def disparity_filter(table, undirected = False, return_self_loops = False):
   # sys.stderr.write("Calculating DF score...\n")
   table = table.copy()
   table_sum = table.groupby(table["src"]).sum().reset_index()
   table_deg = table.groupby(table["src"]).count()["trg"].reset_index()
   table = table.merge(table_sum, on = "src", how = "left", suffixes = ("", "_sum"))
   table = table.merge(table_deg, on = "src", how = "left", suffixes = ("", "_count"))
   table["score"] = 1.0 - ((1.0 - (table["nij"] / table["nij_sum"])) ** (table["trg_count"] - 1))
   table["variance"] = (table["trg_count"] ** 2) * (((20 + (4.0 * table["trg_count"])) / ((table["trg_count"] + 1.0) * (table["trg_count"] + 2) * (table["trg_count"] + 3))) - ((4.0) / ((table["trg_count"] + 1.0) ** 2)))
   if not return_self_loops:
      table = table[table["src"] != table["trg"]]
   if undirected:
      table["edge"] = table.apply(lambda x: "%s-%s" % (min(x["src"], x["trg"]), max(x["src"], x["trg"])), axis = 1)
      table_maxscore = table.groupby(by = "edge")["score"].max().reset_index()
      table_minvar = table.groupby(by = "edge")["variance"].min().reset_index()
      table = table.merge(table_maxscore, on = "edge", suffixes = ("_min", ""))
      table = table.merge(table_minvar, on = "edge", suffixes = ("_max", ""))
      table = table.drop_duplicates(subset = ["edge"])
      
      # *** CORRECTION APPLIED HERE ***
      table = table.drop("edge", axis=1)
      table = table.drop("score_min", axis=1)
      table = table.drop("variance_max", axis=1)
      # *******************************
      
   return table[["src", "trg", "nij", "score", "variance"]]

def high_salience_skeleton(table, undirected = False, return_self_loops = False):
   sys.stderr.write("Calculating HSS score...\n")
   table = table.copy()
   table["distance"] = 1.0 / table["nij"]
   nodes = set(table["src"]) | set(table["trg"])
   G = nx.from_pandas_edgelist(table, source = "src", target = "trg", edge_attr = "distance", create_using = nx.DiGraph())
   cs = defaultdict(float)
   for s in nodes:
      pred = defaultdict(list)
      dist = {t: float("inf") for t in nodes}
      dist[s] = 0.0
      Q = defaultdict(list)
      for w in dist:
         Q[dist[w]].append(w)
      S = []
      while len(Q) > 0:
         v = Q[min(Q.keys())].pop(0)
         S.append(v)
         for _, w, l in G.edges(nbunch = [v,], data = True):
            new_distance = dist[v] + l["distance"]
            if dist[w] > new_distance:
               Q[dist[w]].remove(w)
               dist[w] = new_distance
               Q[dist[w]].append(w)
               pred[w] = []
            if dist[w] == new_distance:
               pred[w].append(v)
         while len(S) > 0:
            w = S.pop()
            for v in pred[w]:
               cs[(v, w)] += 1.0
         Q = defaultdict(list, {k: v for k, v in Q.items() if len(v) > 0})
   table["score"] = table.apply(lambda x: cs[(x["src"], x["trg"])] / len(nodes), axis = 1)
   if not return_self_loops:
      table = table[table["src"] != table["trg"]]
   if undirected:
      table["edge"] = table.apply(lambda x: "%s-%s" % (min(x["src"], x["trg"]), max(x["src"], x["trg"])), axis = 1)
      table_maxscore = table.groupby(by = "edge")["score"].sum().reset_index()
      table = table.merge(table_maxscore, on = "edge", suffixes = ("_min", ""))
      table = table.drop_duplicates(subset = ["edge"])
      table = table.drop("edge", 1)
      table = table.drop("score_min", 1)
      table["score"] = table["score"] / 2.0
   return table[["src", "trg", "nij", "score"]]

In [ ]:
edge_betweenness = nx.edge_betweenness_centrality(G_und, normalized=False)

def normalize_to_range(data, min_new, max_new):
    min_old = min(data)
    max_old = max(data)
    
    new_range = max_new - min_new
    old_range = max_old - min_old
    
    scaled_data = []

    for i in data:
        scaled_data.append(min_new + (i - min_old) * new_range / old_range)
    
    return scaled_data


keys = list(edge_betweenness.keys())
raw_scores = list(edge_betweenness.values())

normalized_weights = normalize_to_range(raw_scores, 1, 100)

In [ ]:
edge_table = pd.DataFrame({
    'src': [edge[0] for edge in keys],
    'trg': [edge[1] for edge in keys],
    'nij': normalized_weights
})

result_filter = disparity_filter(edge_table, undirected=True)

In [ ]:
other_normalized_weights = [1/x for x in normalized_weights]
edge_table2 = pd.DataFrame({
    'src': [edge[0] for edge in keys],
    'trg': [edge[1] for edge in keys],
    'nij': other_normalized_weights
})

result_filter2 = disparity_filter(edge_table, undirected=True)

In [ ]:
result_filter.drop(result_filter[result_filter.score < 0.75].index, inplace=True)

G_backbone = nx.from_pandas_edgelist(
    result_filter,
    source='src',
    target='trg',
    create_using=nx.Graph()  # Use nx.Graph() for the undirected result
)

largest_cc = max(nx.connected_components(G_backbone), key=len)
G_backbone = G_backbone.subgraph(largest_cc)

pose = nx.forceatlas2_layout(G_backbone, max_iter=1000)

In [ ]:
result_filter2.drop(result_filter2[result_filter2.score < 0.75].index, inplace=True)

G_backbone2 = nx.from_pandas_edgelist(
    result_filter2,
    source='src',
    target='trg',
    create_using=nx.Graph()  # Use nx.Graph() for the undirected result
)

largest_cc2 = max(nx.connected_components(G_backbone2), key=len)
G_backbone2 = G_backbone2.subgraph(largest_cc2)

pose2 = nx.forceatlas2_layout(G_backbone2, max_iter=1000)

In [ ]:
node_degrees = dict(G_backbone.degree())
node_sizes = [node_degrees[n] for n in G_backbone.nodes()]
max_degree = max(node_sizes)
scaled_sizes = [100 * (v / max_degree) for v in node_sizes] 

plt.figure(figsize=(8, 8))

# plt.subplot(1,3,1)
nx.draw_networkx_nodes(
    G_backbone, 
    pos=pose, 
    node_size=node_sizes,
    alpha=1.0
)
nx.draw_networkx_edges(
    G_backbone, 
    pos=pose, 
    edge_color='gray',  
    width=0.5,
    alpha=0.3
)
plt.title("Backbone of the network with with weights proportional to edge betweeness centrality")
plt.show()

plt.figure(figsize=(8, 8))

node_degrees2 = dict(G_backbone2.degree())
node_sizes2 = [node_degrees2[n] for n in G_backbone2.nodes()]
max_degree2 = max(node_sizes2)
scaled_sizes2 = [100 * (v / max_degree2) for v in node_sizes2] 
# plt.subplot(1,3,2)
nx.draw_networkx_nodes(
    G_backbone2, 
    pos=pose2, 
    node_size=node_sizes2,
    alpha=1.0
)
nx.draw_networkx_edges(
    G_backbone2, 
    pos=pose2, 
    edge_color='gray',  
    width=0.5,
    alpha=0.3
)
plt.title("Backbone of the network with with weights proportional to 1/edge betweeness centrality")

plt.show()

# Part 2: Genres and communities and plotting 

**We extracted the genre of all musicians with help of the infoboxes of Wikipedia. We start investigating the genres my calculating the number of nodes for which we could find a genre, the average number of genres per node, the total number of distinct genres, and a histogram showing artist counts for the top 15 genres.**

In [ ]:
# As suggested in the part 2 of week 5, we used an LLM to extract the genres and report on the statistics, with extensive testing and revisions to ensure everything works as expected

def report_genre_stats(artist_genres_map: Dict[str, List[str]]):
    """
    Calculates and prints genre statistics, then displays a bar chart.
    """
    print("\n--- Genre Statistics ---")
    
    # 1. Number of nodes with genres
    num_nodes_with_genres = len(artist_genres_map)
    if num_nodes_with_genres == 0:
        print("No genres found for any artists.")
        return
        
    print(f"Nodes with genres found: {num_nodes_with_genres}")

    # 2. Average number of genres per node
    total_genres_assigned = sum(len(genres) for genres in artist_genres_map.values())
    avg_genres = total_genres_assigned / num_nodes_with_genres
    print(f"Average genres per node: {avg_genres:.2f}")

    # 3. Total number of distinct genres
    all_genres_list = [genre for genres in artist_genres_map.values() for genre in genres]
    distinct_genres = set(all_genres_list)
    num_distinct_genres = len(distinct_genres)
    print(f"Total distinct genres:   {num_distinct_genres}")

    # 4. Histogram for top 15 genres
    
    genre_counts = Counter(all_genres_list)
    top_15_genres = genre_counts.most_common(15)
    
    if not top_15_genres:
        print("No genre data to display.")
        return

    # Unzip the data for plotting
    # We reverse the lists [::-1] so the highest bar is at the top
    genres = [genre for genre, count in top_15_genres][::-1]
    counts = [count for genre, count in top_15_genres][::-1]

    # Create horizontal bar plot
    plt.figure(figsize=(10, 8))
    plt.barh(genres, counts)
    plt.xlabel('Number of Artists')
    plt.ylabel('Genre')
    plt.title('Top 15 Most Common Genres')
    
    # Add counts to the end of the bars
    for i, v in enumerate(counts):
        plt.text(v + 0.5, i, str(v), color='blue', va='center')

    plt.tight_layout()  # Adjust plot to prevent label overlap
    plt.show()          # Display the plot

report_genre_stats(genre_dict)

We find that almost all of the nodes have at least one genre, which allows us to maybe use it as a good measurement for detecting communities in the network. However, each artist has an average of 3.75 genres, so the question is which one to choose.

In [ ]:
# Again used the help of an LLM to calculate the modality

def calculate_modularity(G, partition):

    nodes_in_partition = set(partition.keys())
        
    G_partitioned = G.subgraph(nodes_in_partition).copy()
    
    L = G_partitioned.number_of_edges()

    two_L = 2.0 * L
    
    # 3. Calculate Modularity using Equation 9.12
    # M = Σ_c [ (L_c / L) - (k_c / 2L)^2 ]
    
    communities = {}
    for node, comm in partition.items():
        if comm not in communities:
            communities[comm] = []
        communities[comm].append(node)
    
    degrees = dict(G_partitioned.degree())
    
    modularity_sum = 0.0
    for comm_name, nodes_in_comm in communities.items():

        comm_subgraph = G_partitioned.subgraph(nodes_in_comm)
        L_c = comm_subgraph.number_of_edges()
        
        k_c = 0.0
        for node in nodes_in_comm:
            k_c += degrees[node]
            
        term_1 = L_c / L
        term_2 = (k_c / two_L)**2
        
        modularity_sum += (term_1 - term_2)
        
    M = modularity_sum
        
    return len(communities), M


def calculate_genre_modularity(artist_genres_map: Dict[str, List[str]], which_mode):    
    nodes_with_genres = set(artist_genres_map.keys())
    nodes_in_graph = set(G_und.nodes())
    
    valid_nodes = nodes_with_genres.intersection(nodes_in_graph)

    G_filtered = G_und.subgraph(valid_nodes).copy()

    partition = {}
    for node in G_filtered.nodes():
        if artist_genres_map.get(node):
            if which_mode == 0:
                first_genre = artist_genres_map[node][0]
            elif which_mode == 1:
                temp = 0
                if len(artist_genres_map[node]) > 1:
                    temp = 1
                first_genre = artist_genres_map[node][temp]
            elif which_mode == 2:
                first_genre = artist_genres_map[node][random.randint(0, len(artist_genres_map[node])-1)]
            else:
                temp = 0
                if len(artist_genres_map[node]) > 1 and artist_genres_map[node][0] == "hard rock":
                    temp = 1
                first_genre = artist_genres_map[node][temp]
            partition[node] = first_genre
    
    return calculate_modularity(G_filtered, partition)
    


num_comm, modula = calculate_genre_modularity(genre_dict, 0)
print(f"When we use the first genre as a variable for community detection we get {num_comm} unique communities, and a modularity measure of {modula:.4f}.")
num_comm, modula = calculate_genre_modularity(genre_dict, 1)
print(f"\nWhen we use the second genre as a variable for community detection (if a second genre exists, otherwise the first genre) we get {num_comm} unique communities, and a modularity measure of {modula:.4f}.")
num_comm, modula = calculate_genre_modularity(genre_dict, 2)
print(f"\nWhen we use a random genre as a variable for community detection we get {num_comm} unique communities, and a modularity measure of {modula:.4f} (with seed 42).") 


Overall we can see that the genre is not great for community detection. All modularity measures positive but close to 0, so we are in between a suboptimal partition and a single community.

In [ ]:
def calculate_modularity_list(G, partition):

    nodes_in_partition = [x for y in partition for x in y]

    G_partitioned = G.subgraph(nodes_in_partition).copy()
    
    L = G_partitioned.number_of_edges()

    two_L = 2.0 * L
    
    # 3. Calculate Modularity using Equation 9.12
    # M = Σ_c [ (L_c / L) - (k_c / 2L)^2 ]
    
    degrees = dict(G_partitioned.degree())
    
    modularity_sum = 0.0
    for nodes_in_comm in partition:

        comm_subgraph = G_partitioned.subgraph(nodes_in_comm)
        L_c = comm_subgraph.number_of_edges()
        
        k_c = 0.0
        for node in nodes_in_comm:
            k_c += degrees[node]
            
        term_1 = L_c / L
        term_2 = (k_c / two_L)**2
        
        modularity_sum += (term_1 - term_2)
        
    M = modularity_sum
        
    return len(partition), M

louv_comm = nx.community.louvain_communities(G_und, seed=10)    
num_comm, modula = calculate_modularity_list(G_und, louv_comm)

print(f"The buildin louvain-algorithm has a far better community detection, and for seed 10 it calculates {num_comm} communities which have a modalrity score of {modula:.4f}.")

**To investige further how good the genres are for community detection we calculate now the confusion matrix $D$**

In [ ]:
louv_comm = sorted(louv_comm, key=lambda x:len(x), reverse=True)

all_genres_list = [genre for genres in genre_dict.values() for genre in genres]
    
top_7_genres = [x for x,_ in Counter(all_genres_list).most_common(7)]

D = np.zeros((len(louv_comm),len(top_7_genres)))
for i in range(len(louv_comm)):
    for art in louv_comm[i]:
        for j in range(len(top_7_genres)):
            if art in genre_dict:
                if top_7_genres[j] in genre_dict[art]:
                    D[i][j] += 1

print(pd.DataFrame(D, columns=top_7_genres, index=range(1,6)))

We see that the genres could maybe be good for community detection, but they also have some noise. We see e.g. that heavy metal is heavily present in community 3, rock, blues rock, and alternative rock have clear spikes in two of the communities. Hard rock on the other hand is present in all of the communities, which makes sense since this is the most common genre.

In [ ]:
calculate_genre_modularity(genre_dict, 3)

cmap = plt.get_cmap('tab10')

color_map = []

for n in G_und.nodes():
    for i in range(len(louv_comm)):
        if n in louv_comm[i]:
            color_map.append(cmap.colors[i])
            break

pose = nx.forceatlas2_layout(G_und, max_iter=1000)

In [ ]:
plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(
    G_und, 
    pos=pose, 
    node_color=color_map,
    node_size=30,
    alpha=1.0
)
nx.draw_networkx_edges(
    G_und, 
    pos=pose, 
    edge_color='gray',  
    width=0.5,
    alpha=0.2
)
plt.title("Different communities calculated with louvain and displyed with ForceAtlas2")
plt.show()

# Part 3: TF-IDF to understand genres and communities 

* Calculate and visualize TF-IDF for the genres and communities.
* Use the matrix $D$ (Lecture 7, part 2) to dicusss the difference between the word-clouds between genres and communities.


Now let’s get into the topic of TF-IDF. Before explaining the maths and formulas, let’s first start with the intuition. Given a statistic for distinguishing words in a page in a book, we would intuitively look at the most common word. However, this would most likely be a word that is common in ALL pages of the book, words like: the, be, to and of. To fix this, we should not only weigh words by how common they are in the single page of the book, but also by how common they are in ALL of the pages of the book. This is the intuition behind TF-IDF. We calculate how common a word is in a single document, and then we weigh it by how common that word is in the entire corpus.

We can use TF-IDF to identify words that are specific to certain genres. we can then relate these words to certain communities to infer which genres represents which community. Say we use TF-IDF on the page for Hip Hop music, we could find identifying words like "rap, beats, 808s". If we then relate these words to a community found within our network, we can reasonably confidently say that that community represents the Hip Hop genre of music.

Lets now apply TF-IDF to our own dataset. First we have to extract all the genres per artist, as the assignment mentioned to do this using LLM coding, see the code below.

In [23]:
import json
import re
import ast
from pathlib import Path

# --------- Loader helper ---------
def load_entry_file(filepath):
    """Load either valid JSON or a Python dict with single quotes."""
    with open(filepath, "r", encoding="utf-8") as f:
        raw = f.read().strip()

    # Try real JSON first
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        pass

    # Fallback: Python literal (handles single quotes, True/False/None)
    return ast.literal_eval(raw)


# --------- Settings ---------
INPUT_DIR = "Musicians"
EXTS = {".txt", ".json"}

# --------- Regex helpers ---------
GENRE_BLOCK_RE = re.compile(
    r"""
    ^\s*\|\s*genres?\s*=\s*     # allow leading spaces; 'genre' or 'genres'
    (?P<val>                    # capture the value
        (?:.|\n)*?              # lazily consume multiline content
    )
    (?=^\s*\|\s*\w+\s*=|^\s*\}\}|^$)  # stop at next infobox param, close, or file end
    """,
    re.MULTILINE | re.VERBOSE | re.IGNORECASE
)

TEMPLATE_RE = re.compile(r"\{\{(.*?)\}\}", re.S)
WIKILINK_RE = re.compile(r"\[\[([^|\]]+)")
SEPS_RE = re.compile(r"[•·;,\n/]|(?:\s*\*\s*)|\s*\|\s*")

# --------- Cleaning & extraction ---------
def clean_piece(s: str) -> str:
    s = re.sub(r"<ref.*?</ref>", " ", s, flags=re.S)
    s = re.sub(r"\{\{.*?\}\}", " ", s, flags=re.S)
    s = re.sub(r"<.*?>", " ", s, flags=re.S)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_from_template_block(txt: str):
    links = WIKILINK_RE.findall(txt)
    if links:
        return [l.strip() for l in links if l.strip()]
    parts = [p.strip() for p in SEPS_RE.split(txt) if p.strip()]
    return parts

def extract_genres_from_wikitext(wikitext: str):
    """
    Robustly extract 'genre'/'genres' from the Infobox by scanning lines
    and tracking template depth. Crucially: don't strip templates before parsing.
    """
    lines = wikitext.splitlines()
    in_infobox = False
    depth = 0
    captured = []
    collecting = False

    # helpers
    def brace_delta(s: str) -> int:
        # naive but effective for wiki templates
        return s.count("{{") - s.count("}}")

    INFOBOX_START_RE = re.compile(r"\{\{\s*Infobox\s+(?:musical\s+artist|band|artist)\b", re.I)
    PARAM_RE = re.compile(r"^\s*\|\s*(genres?)\s*=\s*(.*)$", re.I)
    NEXT_PARAM_RE = re.compile(r"^\s*\|\s*\w+\s*=")

    for raw in lines:
        line = raw.rstrip("\n")

        if not in_infobox:
            if INFOBOX_START_RE.search(line):
                in_infobox = True
                depth = 1  # we just saw the opening {{Infobox ...
            continue

        # update depth with this line
        depth += brace_delta(line)
        if depth <= 0:
            # infobox closed
            break

        if not collecting:
            m = PARAM_RE.match(line)
            if m:
                collecting = True
                captured.append(m.group(2))  # text after '=' on same line
        else:
            # keep grabbing continuation lines until next param
            if NEXT_PARAM_RE.match(line):
                collecting = False
            else:
                captured.append(line)

    if not captured:
        return []

    # join block but DO NOT remove templates yet
    genre_block = "\n".join(captured)

    # remove obvious noise but keep non-cite templates
    genre_block = re.sub(r"<ref.*?</ref>", " ", genre_block, flags=re.S|re.I)
    genre_block = re.sub(r"\[\[File:.*?\]\]", " ", genre_block, flags=re.S|re.I)
    genre_block = re.sub(r"\{\{cite[^}]*\}\}", " ", genre_block, flags=re.S|re.I)
    # strip common key=value cruft that sometimes leaks into the block
    for k in ["url", "title", "website", "work", "publisher", "access-date", "date",
              "archive-.*", "language"]:
        genre_block = re.sub(rf"\b{k}\s*=\s*.*?(?=\||\n|$)", " ", genre_block, flags=re.I)

    # ---- Extract items ----
    items = []
    templates = TEMPLATE_RE.findall(genre_block)
    if templates:
        for t in templates:
            items.extend(extract_from_template_block(t))
    else:
        links = WIKILINK_RE.findall(genre_block)
        if links:
            items.extend([l.strip() for l in links if l.strip()])
        else:
            # now safe to do a general clean before plain splits
            gb_clean = re.sub(r"\{\{.*?\}\}", " ", genre_block, flags=re.S)  # remove any leftover templates
            gb_clean = clean_piece(gb_clean)
            items.extend([p.strip() for p in SEPS_RE.split(gb_clean) if p.strip()])

    # ---- Final filter/normalize ----
    out, seen = [], set()
    for x in items:
        x = clean_piece(x)
        if not x or len(x) > 60:
            continue
        low = x.lower()
        if any(s in low for s in ["http", "www.", "title=", "access-date", "url=", "cite ", "work="]):
            continue
        if low not in seen:
            seen.add(low)
            out.append(x)

    return out




def load_wikitext_from_entry(entry_dict: dict):
    if "revisions" in entry_dict:
        return entry_dict.get("title", ""), entry_dict["revisions"][0]["*"]
    page = next(iter(entry_dict.values()))
    return page.get("title", ""), page["revisions"][0]["*"]


# --------- Run extraction & print results ---------
rows = []
for path in sorted(Path(INPUT_DIR).iterdir()):
    if path.suffix.lower() not in EXTS or not path.is_file():
        continue

    try:
        data = load_entry_file(path)
        title, wikitext = load_wikitext_from_entry(data)
    except Exception as e:
        print(f"⚠️ Skipping {path.name}: {e}")
        continue

    genres = extract_genres_from_wikitext(wikitext)
    rows.append((title or path.stem, genres))

genre_dict = {artist: genres for artist, genres in rows if genres}

for artist, genres in rows:
    if genres:
        print(f"{artist}: {', '.join(genres)}")
    else:
        print(f"{artist}: (no genre found)")


10 Years (band): Alternative metal, progressive metal, post-grunge, nu&nbsp;metal
10cc: Art rock, art pop, progressive pop, soft rock, pop rock
3 Doors Down: Post-grunge, alternative rock
311 (band): Alternative rock, rap rock, reggae rock, funk rock, funk metal
38 Special (band): Hard rock, southern rock, boogie rock, blues rock
A Perfect Circle: Alternative rock, alternative metal, art rock
ABBA: Pop music, disco, pop rock, Europop
AC/DC: Hard rock, blues rock, rock and roll, Heavy metal music
Accept (band): Heavy metal music
Adam Ant: New wave music, post-punk, alternative rock, dance-rock
Aerosmith: Hard rock, blues rock, Heavy metal music
AFI (band): Punk rock, gothic rock, horror punk
Air Supply: Soft rock, pop rock, pop music, adult contemporary music, Middle of the road (music)
Alanis Morissette: Alternative rock, post-grunge, electronica, grunge, trip hop, indie pop, pop rock
Alice Cooper (band): Hard rock, glam rock, shock rock, psychedelic rock
Alice Cooper: Hard rock, shock

Now that we have all the genres per music artist, we can start to create communities based on words, using TF-IDF. Lets create the TF-IDF matrix for each document in our corpus:

In [ ]:
import math
# First we clean our text, which we can reuse the previous code for:

musicians_cleaned_text = {}

for musician_path in Path("Musicians").glob("*.txt"):
    raw_wikitext = musician_path.read_text(encoding='utf-8')
    cleaned_raw_text = re.sub(r"[^A-Za-z0-9]+", " ", raw_wikitext).strip().lower()
    musicians_cleaned_text[musician_path.stem] = cleaned_raw_text.split()

# SPEEDUP 1: use a set for O(1) membership checks while building vocab
vocab_index = set()
term_counts_per_musician = {}
next_idx = 0  # kept to minimize changes, though not used with a set

for musician in musicians_cleaned_text.keys():
    text = musicians_cleaned_text[musician]
    counts = {}
    for term in text:
        counts[term] = counts.get(term, 0) + 1
        if term not in vocab_index:          # O(1) avg
            vocab_index.add(term)
    term_counts_per_musician[musician] = counts

# Now we have the term counts per musician and the vocabulary index
# Now lets compute the Document Frequency

# SPEEDUP 2: compute DF by updating once per document's unique terms (no V×D loop)
DF = {}
for counts in term_counts_per_musician.values():
    for term in counts.keys():               # unique terms in this doc
        DF[term] = DF.get(term, 0) + 1

# Now lets compute the IDF

IDF = {}
N = len(musicians_cleaned_text)
for term in DF:
    IDF[term] = math.log((N +1)/ (DF[term] + 1)) + 1

# Now with all this data, we can compute the TF-IDF matrix where each row is a 
# document and each column is a term in the vocabulary.

V = len(vocab_index)
TFIDF = {}

for musician, counts in term_counts_per_musician.items():
    tfidf_values = {}
    for term, count in counts.items():
        tf = 1 + math.log(count)
        idf = IDF.get(term, 0)
        tfidf_values[term] = tf * idf
    TFIDF[musician] = tfidf_values

# Optional: L2 normalize each document vector (recommended)
for musician, tfidf_values in TFIDF.items():
    norm = math.sqrt(sum(value ** 2 for value in tfidf_values.values()))
    if norm > 0:
        for term in tfidf_values:
            tfidf_values[term] /= norm
